In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Import the libraries
%reset -f
from __future__ import print_function

import math
import seaborn as sns
import numpy as np
import numpy.linalg as nla
import pandas as pd
import re
import six
from os.path import join
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras_tuner import HyperParameters
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import os
import glob
from tensorflow.data.experimental import load
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import time

start_time = time.time()

In [ ]:
## Function to decode and normalize the images and standardize to RGB
image_size = [224,224]

def decode_image(image_data):
    """Function to decode the image from the .tfrec"""
    ## Converts the raw JPEG file bytes into a 3D tensor, channels=3  indicates RGB, create the shape (height, width, 3), the output is a uint tensor with values 0-255
    image = tf.image.decode_jpeg(image_data, channels=3)
    
    ## Resize all images to the image size specified above [512,512], using bilinear method to smooth the images for efficient processing
    image = tf.image.resize(image, image_size, method = "bilinear")
    
    ## Converts the uint to float32, then normalizes the inputs by dividing by the number of pixel values 255
    ## Removed /255 because EfficientNet expects input [0-255] not [0-1]
    image = tf.cast(image, tf.float32) 
    
    ## Takes the image size defined above this function and reshapes it to be the image size [height, width, 3]
    image = tf.reshape(image, [*image_size,3])
    return image

In [ ]:
def load_dataset(filenames, labeled=True):
    """Load TFRecord dataset from filenames"""
    ## Creates a dataset that reads the files, AUTOTUNE processes them simultneously and TF optimizes the number of readers
    ## Creates a dataset of the raw binary .tfrec examples
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.AUTOTUNE)

    ## dataset.map applies the read_labeled_tfrec function to each example, AUTOTUNE processes them simultneously and TF optimizes the number of readers
    ## Transforms the raw data to (image_tensor, label_int) pairs
    if labeled:
        dataset = dataset.map(read_labeled_tfrec, num_parallel_calls=tf.data.AUTOTUNE)
    else:
        dataset = dataset.map(read_unlabeled_tfrec, num_parallel_calls=tf.data.AUTOTUNE)       
    return dataset

In [ ]:
## Function to return an image, label pair for the training and validation sets
def read_labeled_tfrec(input_example):
    """Read and parse the labeled .tfrec"""
    ## Tells Tensorflow how to interpret the binary .tfrec data, "image" tells TF to expect binary jppeg bytes, "class" tells TF to expect integer labels (the flower labels)
    labeled_tfrec_format = { 
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64)
    }
    ## Parses the input_example using the format specified above
    ## Takes raw binary data (input_example) and uses the labeled_tfrec_format to return a dictionary {image bytes, flower label}
    input_example = tf.io.parse_single_example(input_example, labeled_tfrec_format)
    
    ## Process the image - Takes the JPEG bytes from the example image, and normalizes them to a [512,512,3] tensor using the decode_image function
    image = decode_image(input_example["image"])
    
    ## Process the label - input_example['class'] is the flower class ID
    label = tf.cast(input_example['class'], tf.int32)
    return image, label

In [ ]:
## Function to return an image without labels for the test set
def read_unlabeled_tfrec(input_example):
    """Read and parse the unlabeled .tfrec"""
    unlabeled_tfrec_format = { 
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string)
    }    
    input_example = tf.io.parse_single_example(input_example, unlabeled_tfrec_format)
    
    ## Process the image - Takes the JPEG bytes from the example image, and normalizes them to a [512,512,3] tensor using the decode_image function
    image = decode_image(input_example["image"])
    
    ## Process the label - input_example['id'] is the image ID
    image_id = input_example['id']
    return image, image_id

In [ ]:
## Get filenames for 224x224 images only
folder = 'tfrecords-jpeg-224x224'
train_files = tf.io.gfile.glob(f"/kaggle/input/tpu-getting-started/{folder}/train/*.tfrec")
val_files = tf.io.gfile.glob(f"/kaggle/input/tpu-getting-started/{folder}/val/*.tfrec")
test_files = tf.io.gfile.glob(f"/kaggle/input/tpu-getting-started/{folder}/test/*.tfrec")

## Create train, validation and test data set
train_dataset = load_dataset(train_files, labeled=True)
validation_dataset = load_dataset(val_files, labeled=True)
test_dataset = load_dataset(test_files, labeled=False)

In [ ]:
## Shuffling the training and validation sets

## Sets random seed for reproducability
tf.random.set_seed(42)

## Set shuffle buffer to set how many are shuffled at once
shuffle_buffer = 500

## Shuffle the training set
train_dataset = train_dataset.shuffle(shuffle_buffer, seed=42, reshuffle_each_iteration=False)

## Shuffle the validation set
validation_dataset = validation_dataset.shuffle(shuffle_buffer, seed=42, reshuffle_each_iteration=False)

## No shuffling of test data as it's not needed

In [ ]:
## Set hyperparameters
## Sets the batch to 32 as the number of images to be processed at a time durin one forward/backward pass through the model, 32 is a default
## Larger batches take more memory and are more stable but if they batch is too large the model may generalize poorly
BATCH_SIZE = 32
## Tensorflow optimization that automatically determines the most optimal number of parallel processes
AUTO = tf.data.AUTOTUNE
## Sets the number of classes to the number of flower categories
NUM_CLASSES = 104

## Prepare the datasets for training by grouping a batch based on the size set above, and pre-fetches the next batch while the current batch is being processed
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(AUTO)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(AUTO)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(AUTO)

In [ ]:
# ================================
# Optimizer & Learning Rate
# ================================
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.optimizers import Adam

# Hyperparameters
initial_lr = 1e-3
BATCH_SIZE = 32
NUM_TRAIN_IMAGES = 12752  # Update this if different

# Compute steps per epoch
steps_per_epoch = NUM_TRAIN_IMAGES // BATCH_SIZE

# Cosine Decay Schedule over 80 epochs
lr_schedule = CosineDecay(initial_lr, decay_steps=steps_per_epoch * 80)

# Optimizer
optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
# ================================
# Define Updated DenseNet121 Model
# ================================
def create_model():
    base_model = tf.keras.applications.DenseNet121(
        include_top=False,
        weights="/kaggle/input/pretrained-weights/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
        input_shape=(224, 224, 3),
        pooling='avg'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomContrast(0.1),
        base_model,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    return model, base_model

In [ ]:
# ================================
# Custom Callback for Partial Unfreezing
# ================================
class UnfreezeCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_model, unfreeze_at=5):
        super().__init__()
        self.base_model = base_model
        self.unfreeze_at = unfreeze_at

    def on_epoch_begin(self, epoch, logs=None):
        if epoch == self.unfreeze_at:
            print(f"\nUnfreezing top 100 layers of DenseNet121 at epoch {epoch}")
            self.base_model.trainable = True
            for layer in self.base_model.layers[:-100]:
                layer.trainable = False

In [ ]:
# ================================
# Model Compilation & Training
# ================================
model, base_model = create_model()

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  # removed label_smoothing
    metrics=['accuracy']
)


callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    UnfreezeCallback(base_model, unfreeze_at=5),
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True),
]

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=80,
    callbacks=callbacks
)

In [ ]:
# ================================
# Plot Accuracy and Loss
# ================================
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# ================================
# Save the Model
# ================================
val_loss, val_acc = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

model.save("Samantha_Townsend_flower_densenet121_final.h5")
print("Model saved as Samantha_Townsend_flower_densenet121_final.h5")

In [ ]:
# ================================
# Evaluate and Predict
# ================================
val_loss, val_acc = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

# Predict test labels
predictions = model.predict(test_dataset, verbose=1)
predicted_labels = tf.argmax(predictions, axis=1).numpy()

In [ ]:
# ================================
# Evaluate and Predict
# ================================
predictions = model.predict(test_dataset, verbose=1)
predicted_labels = tf.argmax(predictions, axis=1).numpy()

In [ ]:
# ================================
# Prepare Submission
# ================================

import tensorflow as tf
import pandas as pd
import numpy as np

# === Load your trained model ===
model = tf.keras.models.load_model("Samantha_Townsend_flower_densenet121_final.h5")
print("Model loaded successfully.")

# === Load sample_submission.csv to get correct ID order ===
submission_template = pd.read_csv("/kaggle/input/tpu-getting-started/sample_submission.csv")
test_ids = submission_template["id"].tolist()

# === Define test TFRecord loading function ===
def parse_image(example):
    feature_description = {
        "id": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.decode_jpeg(example["image"], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return example["id"], image

# === Load test dataset (unlabeled) ===
test_files = tf.io.gfile.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/test/*.tfrec")
raw_test_ds = tf.data.TFRecordDataset(test_files, num_parallel_reads=tf.data.AUTOTUNE)
test_ds = raw_test_ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)

# === Store images in a dict keyed by ID for matching ===
test_image_map = {}
for img_id, img in test_ds:
    test_image_map[img_id.numpy().decode()] = img.numpy()

# === Create batch input in the order of submission template ===
ordered_images = np.stack([test_image_map[i] for i in test_ids])
print(f"Loaded {len(ordered_images)} test images in order.")

# === Make predictions ===
pred_probs = model.predict(ordered_images, batch_size=32, verbose=1)
pred_labels = np.argmax(pred_probs, axis=1)

# === Create final submission ===
submission_df = pd.DataFrame({
    "id": test_ids,
    "label": pred_labels
})
submission_df.to_csv("submission.csv", index=False)
print("submission.csv saved successfully!")

